In [62]:
#!python -m spacy download pt_core_news_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests

### Specify Stardog connection details

In [2]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [3]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [4]:
query = """
SELECT ?thesis ?abstract (lang(?abstract) AS ?lang) WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/abstract> ?abstract.
  FILTER (langMatches(lang(?abstract),"pt"))
}
LIMIT 100
"""
#  FILTER (langMatches(lang(?abstract),"en"))
# FILTER (lang(?abstract) NOT IN("pt", "en"))

csv_results = conn.select(query, content_type='text/csv')
thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
thesis_abstract


,thesis,abstract,lang
0,tag:stardog:api:com_a_condicao_de_servir_gratu...,A presente pesquisa parte da emancipação de es...,pt
1,tag:stardog:api:educando_libertos_escravizados...,Esta tese parte da trajetória de Cincinato Fra...,pt
2,tag:stardog:api:marambaia_historia_memoria_e_d...,A presente tese se desdobra em dois objetivos ...,pt
3,tag:stardog:api:mulheres_negras_politicas_iden...,O objetivo do presente estudo é conhecer as in...,pt
4,tag:stardog:api:os_sentidos_da_liberdade_traje...,Esta tese acompanhará trajetórias individuais ...,pt
...,...,...,...
95,tag:stardog:api:reforma_agraria_e_preco_justo_...,Dissertação (mestrado)—Universidade de Brasíli...,pt
96,tag:stardog:api:o_trabalho_domestico_e_a_segun...,Dissertação (mestrado)—Universidade de Brasíli...,pt
97,tag:stardog:api:outras_faces_do_abolicionismo_...,"Tese (doutorado)—Universidade de Brasília, Ins...",pt
98,tag:stardog:api:para_depois_da_escravidao_os_p...,O objetivo desta pesquisa é realizar uma refle...,pt


In [5]:
#nlp_en = spacy.load("en_core_web_trf")
nlp_pt = spacy.load("pt_core_news_lg")

instances_dic = {}

for n in range(len(thesis_abstract)):

    doc_pt = nlp_pt(thesis_abstract['abstract'][n])
    persons = []
    gpes = []
    orgs = []
    for ent in doc_pt.ents:
        if ent.label_ == "PER": #"PERSON":
            persons.append(ent)
        if ent.label_ == "LOC": #"GPE":
            gpes.append(ent)
        if ent.label_ == "ORG":
            orgs.append(ent)

    instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'ORG':orgs}
    
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

#for ent in doc_pt.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [6]:
#total = 0
#total_set = 0
#for i in instances_dic:

#    if instances_dic[i]['PER'] != []:
#        total = total + len(instances_dic[i]['PER'])
#        total_set = total_set + len(set(instances_dic[i]['PER']))
#        print (instances_dic[i]['PER'])
#print (total, total_set)


In [7]:
# Para uma lista de entidades verificar se são a mesma instância e qual o termo mais comum para representá-las

def entidades_consolidadas(list_ent, th):
    
    if list_ent == []:
        return None

    # lista com os vetores das entidades
    list_vectors = np.array([i.vector for i in list_ent])

    #clusterizando os vetores de acordo com threshold th
    clustering = DBSCAN(eps=1-th, min_samples=1, metric='cosine').fit(list_vectors)

    #processnado os clusters 
    ents_dic ={}

    for i in set(clustering.labels_):
        clus_index = np.where(clustering.labels_ == i)[0]
        clus_ent = np.take(np.array(list_ent, dtype="object"), clus_index)

        label = []
        vec = []
        for ent in clus_ent:
            vec.append(ent.vector)
            label.append(ent.text)

        ents_dic[max(set(label), key=label.count)] = {'labels': list(set(label)), 'vector': np.average(vec, axis=0)}

    return ents_dic



In [8]:
th = 0.75
#list_ent = instances_dic['vivencias_e_experiencias_associativas_negras_em_bagers_no_posabolicao_imprensa_carnaval_e_clubes_sociais_negros_na_fronteira_sul_do_brasil_19131980']['LOC']
#list_ent = instances_dic['com_a_condicao_de_servir_gratuitamente_a_mim_ou_a_meus_herdeiros__alforrias_contratos_e_experiencias_de_trabalho_de_libertos_porto_alegre_1884__1888']['LOC']

for key in instances_dic:
    for type_ent in instances_dic[key]:
        list_ent = instances_dic[key][type_ent]
        instances_dic[key][type_ent] = entidades_consolidadas(list_ent, th)
#entidades_consolidadeas(list_ent, th)


Coletando dados da Wikidata

In [9]:
people_labels = []

for key in instances_dic:
    #for type_ent in instances_dic[key]:
    type_ent = 'PER'
    list_ent = instances_dic[key][type_ent]
    if list_ent != None:
        for ent in list_ent:
            people_labels = people_labels + list_ent[ent]['labels']

dbpedia_people = []

for per in set(people_labels):

    query = """     
        SELECT ?person ?abstract
        WHERE{
        ?person rdfs:label '""" + per + """'@pt.
        ?person a dbo:Person.
        ?person dbo:abstract ?abstract.
        FILTER (langMatches(lang(?abstract),"pt"))
        }
        """

    url = 'http://dbpedia.org/sparql'

    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    if data['results']['bindings'] != []:
        dbpedia_people.append((data['results']['bindings'][0]['person']['value'], data['results']['bindings'][0]['abstract']['value']))



In [12]:
dbpedia_people

[('http://dbpedia.org/resource/Deodoro_da_Fonseca',
  'Manuel Deodoro da Fonseca (Alagoas da Lagoa do Sul, 5 de agosto de 1827 – Rio de Janeiro, 23 de agosto de 1892) foi um militar e político brasileiro, primeiro presidente do Brasil e uma das figuras centrais da Proclamação da República no país. Após cursar artilharia na Escola Militar do Rio de Janeiro entre 1843 e 1847, participou de algumas campanhas militares durante o Império, dentre as quais se destacam a Revolução Praieira, o cerco de Montevidéu e a Guerra do Paraguai. Em 1885, tornou-se vice-presidente da província de São Pedro do Rio Grande do Sul; no ano seguinte, com a renúncia do então presidente Barão de Lucena, Deodoro torna-se presidente do Rio Grande. Devido ao seu envolvimento no que ficou conhecido como a Questão Militar — embates entre as forças armadas e o governo civil imperial, — retornou ao Rio de Janeiro. Nomeado para o comando militar da província de Mato Grosso em 1888, exonerou-se do cargo no ano seguinte, 

In [107]:
query = """     
SELECT ?person ?abstract
WHERE{
?person rdfs:label 'José do Patrocínio'@pt.
?person a dbo:Person.
?person dbo:abstract ?abstract.
FILTER (langMatches(lang(?abstract),"pt"))
}
"""

In [108]:
import requests

url = 'http://dbpedia.org/sparql'

r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [121]:
data['results']['bindings'][0]['person']['value']
data['results']['bindings'][0]['abstract']['value']

'José Carlos do Patrocínio (Campos dos Goytacazes, 9 de outubro de 1853 — Rio de Janeiro, 29 de janeiro de 1905) foi um farmacêutico, jornalista, escritor, orador e ativista político brasileiro. Destacou-se como uma das figuras mais importantes do movimento abolicionista no país. Foi também idealizador da Guarda Negra, que era formada por negros e ex-escravos, sendo vanguarda do movimento negro no Brasil e formada para proteger família imperial brasileira contra a aristocracia e os militares.'